In [56]:
import pandas as pd
from hmmlearn.hmm import GaussianHMM
import numpy as np
from sklearn.metrics import confusion_matrix
import itertools
import random
import os

In [5]:



path = "../../dataChallenge/"
name = "Scenario03-01/groundTruth.txt"

data = pd.read_csv(path+name, sep = " ", header = None)
print(len(data))
data.head()     

7722


,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,1,2,1,286,178,290,208,268,208,264,...,318,376,282,174,290,174,276,178,298,178
1,1,3,0,414,150,452,182,438,182,438,...,-1,-1,416,146,418,144,-1,-1,442,142
2,1,1,0,188,152,152,190,134,190,126,...,202,426,182,146,-1,-1,166,148,-1,-1
3,2,2,1,286,178,290,208,268,208,264,...,318,376,282,174,290,174,276,178,298,178
4,2,3,0,414,152,450,180,436,182,438,...,-1,-1,-1,-1,418,144,-1,-1,440,142


In [7]:
nr_of_people = data.iloc[:,1].max()
data_list = [data[data.iloc[:,1]==i+1] for i in xrange(nr_of_people)]
print(data_list)


[        0   1   2    3    4    5    6    7    8    9  ...    29   30   31  \
2        1   1   0  188  152  152  190  134  190  126 ...   202  426  182   
5        2   1   0  188  152  152  190  134  190  126 ...   202  426  182   
8        3   1   0  188  152  152  190  134  190  126 ...   202  426  182   
11       4   1   0  188  152  152  190  134  190  126 ...   200  426  182   
14       5   1   0  188  152  152  190  134  190  126 ...   202  426  182   
17       6   1   0  186  152  152  190  134  190  124 ...   202  426  182   
20       7   1   0  186  152  150  190  134  190  124 ...   200  426  182   
23       8   1   0  184  154  150  190  132  190  122 ...   202  428  180   
26       9   1   0  184  152  150  190  132  190  122 ...   200  428  180   
29      10   1   0  182  152  148  190  130  190  120 ...   200  428  178   
32      11   1   0  180  152  148  190  130  190  120 ...   198  428  176   
35      12   1   0  180  152  146  190  128  190  118 ...   198  426  176  

In [36]:
model = GaussianHMM(n_components = 2)

def train_model(model, data_list, nr_of_epochs):
    
    shuff_data_list = data_list.copy()
    for e in range(nr_of_epochs):
        random.shuffle(shuff_data_list)
        for d in shuff_data_list:

            x = np.array(d.iloc[:,3:])
            model.fit(x)
            
            
            
        
# print("transition probabilities: \n {}".format(model.transmat_))
# pred = model.predict(x)
# cm = confusion_matrix(y, pred)
# norm_cm = cm.astype(float)
# for i, row in enumerate(cm.astype(float)):
#     norm_cm[i,:] = row/np.sum(row)
# spr = float(cm[0,0]*cm[1,1]-cm[0,1]*cm[1,0])**2/(cm[0,0]**2+cm[0,1]**2)/(cm[1,0]**2+cm[1,1]**2)
# print(norm_cm)
# print("sin",spr**.5)
    


transition probabilities: 
 [[  9.99562172e-001   4.37828374e-004]
 [  9.16927636e-210   1.00000000e+000]]
[[ 0.90351873  0.09648127]
 [ 0.85221675  0.14778325]]
('sin', 0.065275650632228455)
transition probabilities: 
 [[ 0.99683258  0.00316742]
 [ 0.01675978  0.98324022]]
[[ 0.83530678  0.16469322]
 [ 0.87439024  0.12560976]]
('sin', 0.051966651006738834)
transition probabilities: 
 [[ 0.99736822  0.00263178]
 [ 0.00283522  0.99716478]]
[[ 0.57687012  0.42312988]
 [ 0.6575      0.3425    ]]
('sin', 0.1520232106695163)


In [64]:
def create_data_list(rootdir):
    file_name = "groundTruth.txt"
    data_list = []
    for dir_ in next(iter(os.walk(rootdir)))[1]:
        # print(dir_)
        data = pd.read_csv(rootdir+ dir_+ os.sep + file_name, sep = " ", header = None)
        nr_of_people = data.iloc[:,1].max()
        data_list += [data[data.iloc[:,1]==i+1] for i in xrange(nr_of_people)]
        
    return data_list

print(len(create_data_list("C:\\Users\\sebbe\\OneDrive\\Dokument\\GitHub\\dataChallenge\\")))
        
    
    
    

52


In [ ]:
def devide_into_frames(data):
    frames = []
    frame_count = data.iloc[:,0].max()
    for i in xrange(frame_count):
        frames.append(data[data.iloc[:,0]==i+1])
    return frames
    

frames = devide_into_frames(data)
print("length: {}".format(len(frames)-1))
#print(frames[10])

In [53]:
def unpanda(a):
    return a.values.tolist()

def identify_people(frames):
    hist = []
    prev = [None,None,None]
    for i, frame in enumerate(frames):
        if len(frame) != 3:
            continue
        def dist(A, B):
            if A is None or B is None: return np.inf
            A, B = unpanda(A)[3:], unpanda(B)[3:]
            s = 0.0
            n = 0
            for Ai, Bi in zip(A, B):
                if Ai == -1 or Bi == -1:
                    continue
                s += (Ai - Bi)**2
                n += 1
            return s/n if n != 0 else np.inf    
            #return np.linalg.norm((A-B)[1:])
        def score(perm):
            now = [frame.iloc[i] for i in perm]
            return sum(dist(previ, nowi) for previ, nowi in zip(prev, now))
        
        tmp = [score(i) for i in itertools.permutations(range(3))]
        print(tmp)
        
        perm = min(itertools.permutations(range(3)), key=score)
        hist.append(perm)
        prev = [frame.iloc[i] for i in range(3)]
        print(type(frame))
    return hist
    
ret = identify_people(frames)

[inf, inf, inf, inf, inf, inf]
<class 'pandas.core.frame.DataFrame'>
[1.6825396825396826, 78199.20202020202, 22826.009523809527, 58631.85627705628, 58925.38701298701, 16533.714285714286]
<class 'pandas.core.frame.DataFrame'>
[22681.219047619048, 58927.42857142857, 1.507936507936508, 79011.02222222222, 16589.714285714283, 59353.01904761905]
<class 'pandas.core.frame.DataFrame'>
[0.9682539682539681, 16655.857142857145, 22439.000000000004, 59162.34285714285, 59228.857142857145, 79297.3111111111]
<class 'pandas.core.frame.DataFrame'>
[2.111111111111111, 16657.0, 22218.714285714286, 58779.857142857145, 58942.05714285714, 78848.3111111111]
<class 'pandas.core.frame.DataFrame'>
[1.7533577533577536, 16664.054945054944, 21691.175824175825, 57555.47619047619, 58366.29010989011, 77568.2888888889]
<class 'pandas.core.frame.DataFrame'>
[2.8144078144078146, 16798.274725274725, 21479.32967032967, 58198.5010989011, 57554.77777777778, 77478.48888888888]
<class 'pandas.core.frame.DataFrame'>
[4.11111111

[3.279761904761905, 64893.83333333333, 20248.696428571428, 49145.28571428572, 49189.19642857143, 13195.232142857143]
<class 'pandas.core.frame.DataFrame'>
[3.8154761904761907, 65246.83333333333, 20405.48214285714, 49487.06547619047, 49409.05952380953, 13247.625]
<class 'pandas.core.frame.DataFrame'>
[3.4543650793650795, 65540.88888888889, 20452.98214285714, 49644.130952380954, 49632.66071428571, 13286.375]
<class 'pandas.core.frame.DataFrame'>
[3.2698412698412698, 65590.22222222222, 20449.964285714286, 49674.97619047619, 49671.94047619047, 13310.0]
<class 'pandas.core.frame.DataFrame'>
[2.4623015873015874, 65698.27777777778, 20558.89285714286, 49759.89880952381, 49808.39880952381, 13313.589285714286]
<class 'pandas.core.frame.DataFrame'>
[3.1507936507936507, 66018.72222222222, 20744.553571428572, 50043.76190476191, 50042.72023809524, 13326.357142857143]
<class 'pandas.core.frame.DataFrame'>
[1.4007936507936507, 66067.88888888888, 20812.678571428572, 50084.767857142855, 50114.4345238095

<class 'pandas.core.frame.DataFrame'>
[2.5112745098039215, 70453.73755656107, 21224.800000000003, 53263.825641025636, 53223.2923076923, 14811.091666666667]
<class 'pandas.core.frame.DataFrame'>
[2.7759803921568627, 69892.42533936651, 20916.13333333333, 52872.153846153844, 52721.88717948718, 14788.258333333333]
<class 'pandas.core.frame.DataFrame'>
[4.2710084033613445, 67960.5173453997, 20573.285714285714, 51067.399999999994, 51958.329670329666, 14496.19761904762]
<class 'pandas.core.frame.DataFrame'>
[2.80672268907563, 67394.04298642534, 20235.77142857143, 51635.58388278388, 50476.3, 14484.87619047619]
<class 'pandas.core.frame.DataFrame'>
[3.2549019607843137, 68029.04977375566, 19906.266666666666, 51410.21538461538, 51266.77948717949, 14744.933333333334]
<class 'pandas.core.frame.DataFrame'>
[19466.133333333335, 50770.64615384615, 3.6279411764705882, 67168.96832579185, 14688.475, 50549.30256410257]
<class 'pandas.core.frame.DataFrame'>
[19046.4, 49896.071794871794, 3.6686274509803924,

<class 'pandas.core.frame.DataFrame'>
[10727.125, 3.3472222222222223, 55593.32142857143, 29338.375, 71227.50793650793, 55696.33928571428]
<class 'pandas.core.frame.DataFrame'>
[29119.75, 55298.41071428572, 6.902777777777778, 10626.625, 70781.20634920636, 55222.267857142855]
<class 'pandas.core.frame.DataFrame'>
[4.633333333333333, 70293.78021978022, 28888.008333333335, 54827.17307692308, 54752.65714285715, 10402.675]
<class 'pandas.core.frame.DataFrame'>
[3.3444444444444446, 70206.03418803419, 28587.733333333334, 54741.196153846155, 54414.126923076925, 10364.900000000001]
<class 'pandas.core.frame.DataFrame'>
[5.058333333333334, 70394.61538461538, 28332.908333333333, 54798.42628205128, 54533.3641025641, 10609.325]
<class 'pandas.core.frame.DataFrame'>
[6.4299019607843135, 70509.66515837103, 28118.13333333333, 54786.5641025641, 54596.51282051282, 10761.708333333332]
<class 'pandas.core.frame.DataFrame'>
[4.029411764705882, 70866.76018099548, 27921.6, 55050.3282051282, 54684.902564102566

[1.6313025210084033, 18506.732142857145, 23641.619047619046, 61682.68095238095, 61758.17619047619, 81294.13725490196]
<class 'pandas.core.frame.DataFrame'>
[1.542016806722689, 18493.071428571428, 23657.904761904763, 61796.038095238095, 61645.36666666667, 81291.9705882353]
<class 'pandas.core.frame.DataFrame'>
[1.8813025210084033, 18440.410714285714, 23750.685714285715, 61778.87619047619, 61741.80952380952, 81331.4705882353]
<class 'pandas.core.frame.DataFrame'>
[1.9926470588235294, 18419.160714285717, 23745.933333333334, 61699.97142857143, 61684.91428571429, 81221.78431372548]
<class 'pandas.core.frame.DataFrame'>
[1.3067226890756303, 18388.64285714286, 23684.30476190476, 61576.97142857143, 61567.904761904756, 81073.23529411765]
<class 'pandas.core.frame.DataFrame'>
[1.8529411764705883, 18363.5, 23688.86666666667, 61594.73809523809, 61533.96190476191, 81078.1862745098]
<class 'pandas.core.frame.DataFrame'>
[1.1418067226890756, 18306.696428571428, 23696.961904761905, 61508.8380952381, 6

...

<class 'pandas.core.frame.DataFrame'>
[2030.9915966386554, 683.7058823529412, 14034.866666666667, 10724.866666666667, 7401.0, 5438.285714285714]
<class 'pandas.core.frame.DataFrame'>
[inf, 3863.5117647058823, inf, 8515.833333333334, 12263.0, 4462.125]
<class 'pandas.core.frame.DataFrame'>
[36.39558823529412, 4344.470588235294, 13050.533333333333, 11759.2, 11144.033333333335, 5544.625]
<class 'pandas.core.frame.DataFrame'>
[489.4720588235294, 5568.847058823529, 14028.591666666667, 12743.625, 11393.466666666667, 5029.125]
<class 'pandas.core.frame.DataFrame'>
[29.683473389355743, 5618.766013071896, 15276.521870286575, 12019.962393162392, 14312.203921568627, 5466.561904761905]
<class 'pandas.core.frame.DataFrame'>
[44.03174603174604, 6701.222222222223, 15206.819047619048, 14891.866666666667, 14067.73015873016, 7095.587301587302]
<class 'pandas.core.frame.DataFrame'>
[32.6, 6708.222222222223, 14725.111111111111, 14627.666666666668, 11822.222222222223, 5049.155555555555]
<class 'pandas.core

[8563.284313725491, 57462.802139037434, 7.990196078431373, 33231.657754010695, 73006.07486631017, 57330.22459893048]
<class 'pandas.core.frame.DataFrame'>
[8.84313725490196, 74940.35294117648, 8775.196078431372, 58942.812834224605, 58854.53475935829, 34090.64171122995]
<class 'pandas.core.frame.DataFrame'>
[9002.78431372549, 60250.05347593583, 6.313725490196079, 76752.99465240643, 34901.5614973262, 60400.97326203209]
<class 'pandas.core.frame.DataFrame'>
[9.53921568627451, 35613.5614973262, 9188.598039215687, 61600.12834224599, 61586.0, 78393.50802139039]
<class 'pandas.core.frame.DataFrame'>
[9217.676470588236, 62841.71122994652, 8.735294117647058, 36566.20320855615, 80105.66844919787, 63039.101604278076]
<class 'pandas.core.frame.DataFrame'>
[9.107843137254903, 81937.5294117647, 9359.225490196079, 64377.34759358289, 64423.71122994652, 37513.41176470588]
<class 'pandas.core.frame.DataFrame'>
[7.147058823529411, 84602.03654188948, 9464.79411764706, 66777.22370766489, 66007.91889483065,

[2.815126050420168, 94450.39819004525, 44708.36199095023, 74264.5701357466, 74341.71945701358, 9450.344537815126]
<class 'pandas.core.frame.DataFrame'>
[3.19327731092437, 94538.09049773756, 44713.3665158371, 74433.19457013575, 74321.79185520363, 9506.72268907563]
<class 'pandas.core.frame.DataFrame'>
[1.9831932773109244, 94617.47511312217, 44707.16742081448, 74437.7556561086, 74457.00452488688, 9572.100840336134]
<class 'pandas.core.frame.DataFrame'>
[3.8319327731092434, 94416.17194570135, 44527.94570135747, 74310.52488687783, 74229.592760181, 9599.831932773108]
<class 'pandas.core.frame.DataFrame'>
[3.0, 94321.16742081448, 44280.98642533937, 74210.7330316742, 74091.42081447964, 9703.0]
<class 'pandas.core.frame.DataFrame'>
[4.142857142857143, 94457.4479638009, 44193.93665158371, 74313.29411764706, 74191.73755656109, 9857.789915966387]
<class 'pandas.core.frame.DataFrame'>
[2.2184873949579833, 94623.23981900452, 44175.33031674208, 74416.80542986425, 74329.5294117647, 9949.983193277312]

[5.260504201680672, 101643.74660633484, 40812.37104072398, 78670.88687782804, 78236.52488687783, 14456.55462184874]
<class 'pandas.core.frame.DataFrame'>
[6.840336134453781, 101939.40400775694, 40210.71105365223, 79374.76535229477, 77756.05559146736, 14987.546218487394]
<class 'pandas.core.frame.DataFrame'>
[3.843137254901961, 102024.15966386555, 39566.30252100841, 78771.38655462186, 78373.19327731093, 15557.960784313727]
<class 'pandas.core.frame.DataFrame'>
[3.223529411764706, 103094.73109243697, 39521.092436974795, 79509.84873949579, 79086.44537815126, 15983.69411764706]
<class 'pandas.core.frame.DataFrame'>
[4.847058823529411, 104346.54621848739, 39439.529411764706, 80385.42016806723, 79884.18487394958, 16488.376470588235]
<class 'pandas.core.frame.DataFrame'>
[3.0823529411764703, 105504.2100840336, 39413.38655462185, 81053.34453781512, 80777.89915966387, 16916.729411764707]
<class 'pandas.core.frame.DataFrame'>
[4.031372549019608, 106554.30252100842, 39376.89075630253, 81828.78151

[4.817496229260935, 138456.85897435897, 23654.778280542985, 105620.35434173669, 104411.31221719457, 47924.846800258565]
<class 'pandas.core.frame.DataFrame'>
[23763.719457013573, 105554.26244343891, 18.4841628959276, 139772.92307692306, 48482.011635423405, 106445.90756302522]
<class 'pandas.core.frame.DataFrame'>
[4.824463118580766, 49410.98319327731, 23838.680672268907, 106623.09437621203, 106981.91596638656, 140360.17094017094]
<class 'pandas.core.frame.DataFrame'>
[50253.84033613445, 9.030812324929972, 108072.90627020039, 141678.2564102564, 23816.873949579833, 107667.03361344538]
<class 'pandas.core.frame.DataFrame'>
[4.995331465919701, 51285.487394957985, 142987.37606837606, 109282.31544925662, 108677.3257918552, 23691.773109243695]
<class 'pandas.core.frame.DataFrame'>
[5.579831932773109, 52583.56237879767, 143904.07239819004, 110346.3438914027, 109566.93794440854, 23431.226890756305]
<class 'pandas.core.frame.DataFrame'>
[6.697478991596638, 53586.95798319328, 144817.5746606335, 1

In [31]:
def dist(A, B):
    if A is None or B is None: return np.inf
    s = 0.0
    n = 0
    for Ai, Bi in zip(A, B):
        if Ai == -1 or Bi == -1:
            continue
        s += (Ai - Bi)**2
        n += 1
    return s/n if n != 0 else np.inf 

AS = frames[1]
BS = frames[2]
print(np.array(AS), np.array(BS))

(array([[  2,   2,   1, 286, 178, 290, 208, 268, 208, 264, 242, 266, 270,
        312, 208, 322, 242, 320, 268, 280, 280, 286, 332, 292, 378, 306,
        278, 310, 330, 318, 376, 282, 174, 290, 174, 276, 178, 298, 178],
       [  2,   3,   0, 414, 152, 450, 180, 436, 182, 438, 244, 422, 286,
        464, 180, 474, 244, 444, 286, 436, 296, 422, 366, 422, 430, 462,
        296, 464, 380,  -1,  -1,  -1,  -1, 418, 144,  -1,  -1, 440, 142],
       [  2,   1,   0, 188, 152, 152, 190, 134, 190, 126, 250, 160, 290,
        172, 190,  -1,  -1,  -1,  -1, 152, 300, 146, 376, 134, 442, 186,
        294, 208, 360, 202, 426, 182, 146,  -1,  -1, 166, 148,  -1,  -1]], dtype=int64), array([[  3,   3,   0, 412, 152, 450, 182, 436, 182, 438, 244,  -1,  -1,
        464, 180, 474, 244, 444, 286, 434, 296, 422, 366, 422, 430, 462,
        296, 462, 380,  -1,  -1,  -1,  -1, 416, 144,  -1,  -1, 438, 144],
       [  3,   2,   1, 286, 178, 290, 208, 270, 208, 264, 242, 266, 270,
        312, 208, 322, 242, 320

In [54]:
all(i==(0,1,2) for i in ret)

False

In [55]:
ret


[(0, 1, 2),
 (0, 1, 2),
 (1, 0, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (1, 0, 2),
 (1, 0, 2),
 (0, 1, 2),
 (1, 0, 2),
 (1, 0, 2),
 (0, 1, 2),
 (1, 0, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (1, 0, 2),
 (1, 0, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 2, 1),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0,

In [52]:
frames[1].iloc[0]

0       2
1       2
2       1
3     286
4     178
5     290
6     208
7     268
8     208
9     264
10    242
11    266
12    270
13    312
14    208
15    322
16    242
17    320
18    268
19    280
20    280
21    286
22    332
23    292
24    378
25    306
26    278
27    310
28    330
29    318
30    376
31    282
32    174
33    290
34    174
35    276
36    178
37    298
38    178
Name: 3, dtype: int64

In [47]:
f = frames[1].values.tolist()
f[0]

[2L,
 2L,
 1L,
 286L,
 178L,
 290L,
 208L,
 268L,
 208L,
 264L,
 242L,
 266L,
 270L,
 312L,
 208L,
 322L,
 242L,
 320L,
 268L,
 280L,
 280L,
 286L,
 332L,
 292L,
 378L,
 306L,
 278L,
 310L,
 330L,
 318L,
 376L,
 282L,
 174L,
 290L,
 174L,
 276L,
 178L,
 298L,
 178L]